In [1]:
import backtrader as bt
import pandas as pd
import pandas_ta as ta
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf


In [30]:
from backtrader.studies.contrib.fractal import Fractal

## Source data locally

In [22]:
data_local = pd.read_csv("../data/xauusd_1h.csv", index_col=None)
data_local['datetime'] = pd.to_datetime(data_local['datetime'])
print(data_local.head())

data_feed_local = bt.feeds.PandasData(dataname=data_local, name="xau",
                                      datetime=0, openinterest=-1,
                                      open=1,
                                      high=2,
                                      low=3,
                                      close=4,
                                      volume=5)

data_feeds_local = dict(xau=data_feed_local)

             datetime         open         high          low        close  \
0 2010-01-03 22:00:00  1097.880005  1097.880005  1097.449951  1097.449951   
1 2010-01-03 23:00:00  1097.680054  1100.599976  1095.979980  1095.979980   
2 2010-01-04 00:00:00  1096.010010  1096.869995  1093.449951  1094.839966   
3 2010-01-04 01:00:00  1094.869995  1095.959961  1094.239990  1095.699951   
4 2010-01-04 02:00:00  1095.670044  1099.150024  1095.630005  1098.329956   

   volume  
0       2  
1     919  
2    1596  
3     869  
4    1054  


## Download Data from YF

In [79]:
# Download SPX500 data from Yahoo Finance
yf_params = {'start': '2018-06-01', 'end': '2025-01-01',
             'interval': '1d', 'multi_level_index': False, 'auto_adjust': True}
data_names = [
    # '^RUT', '^IXIC', '^VIX',
    '^DJI', '^GSPC', 'GOOG', 'MSFT', 'AMZN', 'TSLA', 
              'NFLX', 'NVDA', 'AMD', 'KO']

datas = dict()
for name in data_names:
    datas[name] = yf.download(tickers=name, **yf_params)

dt_col = 'Datetime' if yf_params['interval'] == '4h' else 'Date'

for name, data in datas.items():
    data.reset_index(inplace=True)
    data[dt_col] = pd.to_datetime(data[dt_col])
    print(f"{name}: from {data[dt_col].iloc[0]} to {data[dt_col].iloc[-1]}")


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

^DJI: from 2018-06-01 00:00:00 to 2024-12-31 00:00:00
^GSPC: from 2018-06-01 00:00:00 to 2024-12-31 00:00:00
GOOG: from 2018-06-01 00:00:00 to 2024-12-31 00:00:00
MSFT: from 2018-06-01 00:00:00 to 2024-12-31 00:00:00
AMZN: from 2018-06-01 00:00:00 to 2024-12-31 00:00:00
TSLA: from 2018-06-01 00:00:00 to 2024-12-31 00:00:00
NFLX: from 2018-06-01 00:00:00 to 2024-12-31 00:00:00
NVDA: from 2018-06-01 00:00:00 to 2024-12-31 00:00:00
AMD: from 2018-06-01 00:00:00 to 2024-12-31 00:00:00
KO: from 2018-06-01 00:00:00 to 2024-12-31 00:00:00


In [80]:
data_feeds = dict()
for name, data in datas.items():
    data_feeds[name] = bt.feeds.pandafeed.PandasData(
        name=name,
        dataname=data,
        datetime=0,
        openinterest=-1,
        open=1,
        high=2,
        low=3,
        close=4,
        volume=5,
    )

# Start Backtesting

In [73]:

class TriplleMARSIFracStrategy(bt.Strategy):
    params = (
        # Position Management Params
        ('atr_period', 9) ,      # ATR Period
        ('atr_multiplier', 1.5), # ATR multiplier for SL calculation
        ("rrr", 2),  # Risk-Reward Ratio
        # Triple MA Params
        ('fastMA_period', 21),
        ('midMA_period', 50),
        ('slowMA_period', 200),
        # RSI params
        ('rsi_period', 14),
    )
    def __init__(self):
        # initialize the EMA indicator on data feed
        self.fast_ma = bt.ind.SmoothedMovingAverage(period=self.params.fastMA_period)
        self.mid_ma = bt.ind.SmoothedMovingAverage(period=self.params.midMA_period)
        self.slow_ma = bt.ind.SmoothedMovingAverage(period=self.params.slowMA_period)
        self.rsi = bt.ind.RelativeStrengthIndex(period=self.params.rsi_period)
        self.fractal = Fractal()
        self.atr = bt.indicators.AverageTrueRange(period=self.params.atr_period)
        self.order = None

    def _isBulluish(self, idx:int):
        if len(self.data) < idx+1:
            self.log("[Error]: Index out of bounds for `isBullish`")
            return
        return self.data.close[idx] > self.data.open[idx]

    def _isEngulfing(self, idx:int):
        if len(self.data) < idx+1:
            self.log("[Error]: Index out of bounds for `isEnglufing`")
            return
        return self.data.open[idx+1] < self.data.close[idx] if self._isBulluish(idx) \
            else self.data.open[idx+1] > self.data.close[idx]

    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, {order.executed.price:.2f}')
            elif order.issell():
                self.log(f'SELL EXECUTED, {order.executed.price:.2f}')
        self.order = None
    
    def log(self, txt):
        dt = self.data.datetime.date(0)
        print(f'{dt.isoformat()} {txt}')
    
    
    def _sell(self):
        price = self.data.close[0]
        sl_price = price + self.atr * self.params.atr_multiplier
        tp_price = price - np.abs(sl_price - price) * self.params.rrr
        if tp_price > price or tp_price < 0:
            self.log(f'Take profit price {tp_price:.2f} is greater than entry price {price:.2f}, not placing order.')
            return
        if sl_price < price or sl_price < 0:
            self.log(f'Stop loss price {sl_price:.2f} is less than entry price {price:.2f}, not placing order.')
            return
        self.order = self.sell_bracket(
            exectype=bt.Order.Market, 
            price=price,
            stopprice=sl_price,
            limitprice=tp_price,
            stopargs=dict(exectype=bt.Order.Stop),
            limitargs=dict(exectype=bt.Order.Limit),
        )
        self.log(f"Bracket order placed: Entry {price:.2f}, SL {sl_price:.2f}, TP {tp_price:.2f}")        

    def _buy(self):
        price = self.data.close[0]
        sl_price = price - self.atr * self.params.atr_multiplier
        tp_price = price + np.abs(sl_price - price) * self.params.rrr
        if tp_price < price or tp_price < 0:
            self.log(f'Take profit price {tp_price:.2f} is less than entry price {price:.2f}, not placing order.')
            return
        if sl_price > price or sl_price < 0:
            self.log(f'Stop loss price {sl_price:.2f} is greater than entry price {price:.2f}, not placing order.')
            return

        self.order = self.buy_bracket(
            exectype=bt.Order.Market, 
            price=price,
            stopprice=sl_price,
            limitprice=tp_price,
            stopargs=dict(exectype=bt.Order.Stop),
            limitargs=dict(exectype=bt.Order.Limit),
        )
        self.log(f"Bracket order placed: Entry {price:.2f}, SL {sl_price:.2f}, TP {tp_price:.2f}")

    def next(self):
        # if in market position, do nothing, return
        if self.position:
            return


        last_high = self.data.high[0]
        last_low = self.data.low[0]
        if (
            self.fractal.fractal_bearish[0] != 0 and
            self.rsi[0] < 50 and
            last_high < self.fast_ma[0] and
            self.fast_ma[0] < self.mid_ma[0]
            # self.mid_ma[0] < self.slow_ma[0]
        ):
            self._sell()
            
        elif (
            self.fractal.fractal_bullish[0] != 0 and
            self.rsi[0] > 50 and
            last_low > self.fast_ma[0] and
            self.fast_ma[0] > self.mid_ma[0]
            # self.mid_ma[0] > self.slow_ma[0]
        ):
            self._buy()

In [82]:
def log_results(data_name:str, results) -> None:
    print("\n\n" + 20*"=" + f" {data_name} " + 20*"=")
    strat = results[0]
    sharpe_ratio = strat.analyzers.sharperatio.get_analysis()
    drawdown = strat.analyzers.drawdown.get_analysis()
    trade_analysis = strat.analyzers.tradeanalyzer.get_analysis()
    returns = strat.analyzers.returns.get_analysis()
    print(f'Sharpe Ratio: {sharpe_ratio['sharperatio']}')
    print(f'Max Drawdown: {drawdown["max"]["drawdown"]}')
    print(f'Number of Trades: {trade_analysis.total.closed}')
    print(f'Winning Trades: {trade_analysis.won.total}')
    print(f'Losing Trades: {trade_analysis.lost.total}')
    print(f'Average Trade Return: {returns["rnorm"]}')
    print(f'Total Returns: {returns["rtot"]}')
    print(f'Final Balance: {strat.broker.getvalue()}')

In [75]:
def test_strategy(data_feed, strategy:bt.Strategy, strategy_params:dict):
    cerebro = bt.Cerebro()
    # Add the Strategy
    cerebro.addstrategy(strategy,
                        **strategy_params
                        )
    # Add trade size manager (Percent of equity)
    cerebro.addsizer(bt.sizers.PercentSizer, percents=5)

    cerebro.adddata(data_feed)  # Add DataFeed
    # Set cash and commission
    cerebro.broker.setcash(100_000.0)
    cerebro.broker.setcommission(commission=0.001)
    # Add Analyzers for final review
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.01)
    cerebro.addanalyzer(bt.analyzers.DrawDown)
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer)
    cerebro.addanalyzer(bt.analyzers.Returns)

    results = cerebro.run(stdstats=False)
    
    return results, cerebro

In [81]:
strategy_params = dict(
                       atr_period=11,
                       atr_multiplier=1,
                       rrr=2,
)
results = dict()

for name, data_feed in data_feeds.items():
    results[name], cereb = test_strategy(data_feed, TriplleMARSIFracStrategy, strategy_params)
    # cereb.plot(iplot=True)

for name, res in results.items():
    log_results(name, res)

2019-03-19 Bracket order placed: Entry 25987.87, SL 25687.77, TP 26588.08
2019-03-20 BUY EXECUTED, 25745.67
2019-03-21 SELL EXECUTED, 25687.77
2019-03-21 Bracket order placed: Entry 25688.44, SL 25382.19, TP 26300.94
2019-03-22 BUY EXECUTED, 25502.32
2019-03-25 SELL EXECUTED, 25382.19
2019-03-26 Bracket order placed: Entry 25649.56, SL 25324.44, TP 26299.81
2019-03-27 BUY EXECUTED, 25625.59
2019-04-04 SELL EXECUTED, 26384.63
2019-04-04 Bracket order placed: Entry 26213.42, SL 25933.43, TP 26773.39
2019-04-05 BUY EXECUTED, 26424.99
2019-05-07 SELL EXECUTED, 25933.43
2019-06-07 Bracket order placed: Entry 25768.72, SL 25385.86, TP 26534.43
2019-06-10 BUY EXECUTED, 26062.68
2019-06-19 SELL EXECUTED, 26534.43
2019-06-19 Bracket order placed: Entry 26490.16, SL 26170.01, TP 27130.46
2019-06-20 BUY EXECUTED, 26753.17
2019-07-12 SELL EXECUTED, 27332.03
2019-07-12 Bracket order placed: Entry 27139.49, SL 26894.79, TP 27628.90
2019-07-15 BUY EXECUTED, 27359.16
2019-07-31 SELL EXECUTED, 26864.27

In [84]:
res[0].analyzers.drawdown.get_analysis()

AutoOrderedDict([('len', 1203),
                 ('drawdown', 2.8481827191190185),
                 ('moneydown', 2874.699988075896),
                 ('max',
                  AutoOrderedDict([('len', 1203),
                                   ('drawdown', 3.6618441800390387),
                                   ('moneydown', 3695.936833697968)]))])